<a href="https://colab.research.google.com/github/jamesdievas/AnimalProtector/blob/main/PF_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalação de bibliotecas
!pip install tensorflow opencv-python pillow python-dotenv requests pyinaturalist

import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from pyinaturalist import get_observations
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import os
import io
import requests
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from pyinaturalist import get_observations
import unicodedata

import smtplib
from email.mime.text import MIMEText
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from dotenv import load_dotenv

# Caminho para o .env dentro do Drive
env_path = "/content/drive/MyDrive/Faculdade_2025/PF/credentials/.env"

# Carrega o arquivo
load_dotenv(dotenv_path=env_path)

# Carrega as Crediciais do arquivo .env que está na pasta do projeto
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
EMAIL_TO = EMAIL_USER  # pode alterar para outro destinatário

In [ ]:
import os
import requests
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from pyinaturalist import get_observations

# ---------------------------
#  1 Lista selecionada de espécies
# ---------------------------
especies = [
    "arara-azul",
    "onça-pintada",
    "mico-leão-dourado",
    "tamanduá-bandeira",
    "anta"
]

 #Criar dicionário para nomes das pastas
especies_dict = {nome.lower().replace(" ", "_"): nome for nome in especies}

# 2 Função para baixar imagens do iNaturalist
# ============================================================
def baixar_imagens(nome_pasta, nome_cientifico, limite=40):
    os.makedirs(f"data/{nome_pasta}", exist_ok=True)
    try:
        obs = get_observations(taxon_name=nome_cientifico, per_page=limite)
    except Exception as e:
        print(f"[ERRO] Falha ao buscar {nome_cientifico}: {e}")
        return

    count = 0
    for i, record in enumerate(obs.get("results", [])):
        fotos = record.get("photos")
        if fotos:
            url = fotos[0]["url"].replace("square", "original")
            try:
                img = requests.get(url, timeout=10).content
                with open(f"data/{nome_pasta}/{i}.jpg", "wb") as f:
                    f.write(img)
                count += 1
            except Exception:
                pass
    print(f"[OK] {count} imagens baixadas para {nome_pasta}")

# ============================================================
# 3 Baixar imagens
# ============================================================
for pasta, cientifico in especies_dict.items():
    baixar_imagens(pasta, cientifico, limite=40)

# ============================================================
# 4 Preparar dataset TensorFlow
# ============================================================
IMG_HEIGHT, IMG_WIDTH = 120, 120
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size
)

# Salvar classes antes do map
classes = train_ds.class_names

# Normalização
rescale_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (rescale_layer(x), y))
val_ds = val_ds.map(lambda x, y: (rescale_layer(x), y))

print("✅ Dataset pronto com sucesso!")
print("Classes:", classes)

[OK] 39 imagens baixadas para arara-azul
[OK] 39 imagens baixadas para onça-pintada
[OK] 39 imagens baixadas para mico-leão-dourado
[OK] 40 imagens baixadas para tamanduá-bandeira
[OK] 40 imagens baixadas para anta
Found 197 files belonging to 5 classes.
Using 158 files for training.
Found 197 files belonging to 5 classes.
Using 39 files for validation.
✅ Dataset pronto com sucesso!
Classes: ['anta', 'arara-azul', 'mico-leão-dourado', 'onça-pintada', 'tamanduá-bandeira']


In [ ]:

# ============================================================
# 5 Construção do modelo CNN
# ============================================================

num_classes = len(classes)
model = Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Treinamento
epochs = 5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

# Salvar modelo
model.save("pre_models.keras")

# ============================================================
# 6 Classificação em tempo real + email
# ============================================================

# Função para enviar email
def enviar_email(assunto, mensagem):
    msg = MIMEMultipart()
    msg['From'] = EMAIL_USER
    msg['To'] = EMAIL_TO
    msg['Subject'] = assunto
    msg.attach(MIMEText(mensagem, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(EMAIL_USER, EMAIL_PASS)
        server.sendmail(EMAIL_USER, EMAIL_TO, msg.as_string())
        server.quit()
        print("[OK] Email enviado com sucesso.")
    except Exception as e:
        print("[ERRO] Falha ao enviar email:", e)

# Função para classificar imagem
def classificar_imagem(img_array):
    img = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    indice = np.argmax(pred)
    classe = classes[indice]
    return classe, float(pred[0][indice])

# Captura via webcam
def capturar_webcam():
    cap = cv2.VideoCapture(0)
    print("Pressione 'q' para capturar")
    while True:
        ret, frame = cap.read()
        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return frame

# Carregar imagem de arquivo ou URL
def carregar_imagem(path_or_url):
          return cv2.imread(path_or_url)

# Função principal
def processar_imagem(entrada):
    if entrada == "webcam":
        img = capturar_webcam()
    else:
        img = carregar_imagem(entrada)

    classe, confianca = classificar_imagem(img)
    print(f"Classe detectada: {classe} ({confianca*100:.2f}%)")

    if confianca > 0.7:
        mensagem = f"Espécie Ameaçada Detectada: {classe}\nConfiança: {confianca*100:.2f}%"
        enviar_email("Alerta: Espécie Ameaçada Detectada", mensagem)

# ============================================================
# 6 Função de monitoramento contínuo da webcam
# ============================================================

def monitorar_webcam(conf_threshold=0.7):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("[ERRO] Não foi possível abrir a webcam.")
        return

    print("Pressione 'q' para encerrar o monitoramento.")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        # Classificação
        classe, confianca = classificar_imagem(frame)

        # Mostra resultado na imagem
        display_frame = frame.copy()
        cv2.putText(
            display_frame,
            f"{classe} ({confianca*100:.1f}%)",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0) if confianca >= conf_threshold else (0, 0, 255),
            2
        )
        cv2.imshow("Monitoramento - Pressione 'q' para sair", display_frame)

        # Envia email se confiança >= limite
        if confianca >= conf_threshold:
            mensagem = f"Espécie Ameaçada Detectada: {classe}\nConfiança: {confianca*100:.2f}%"
            enviar_email("Alerta: Espécie Ameaçada Detectada", mensagem)
            print(f"[ALERTA] Email enviado para {classe}")

        # Sair ao pressionar 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("✅ Monitoramento encerrado.")

Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 564ms/step - accuracy: 0.1645 - loss: 3.3597 - val_accuracy: 0.3333 - val_loss: 1.5773
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 569ms/step - accuracy: 0.2681 - loss: 1.6007 - val_accuracy: 0.2051 - val_loss: 1.6079
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 470ms/step - accuracy: 0.3217 - loss: 1.4975 - val_accuracy: 0.1795 - val_loss: 1.7031
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 539ms/step - accuracy: 0.4316 - loss: 1.3620 - val_accuracy: 0.3590 - val_loss: 1.4860
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 560ms/step - accuracy: 0.6315 - loss: 1.1274 - val_accuracy: 0.4103 - val_loss: 1.4768


In [ ]:
# ============================================================
# 7 Exemplo de uso
# ============================================================
#processar_imagem("webcam")
#monitorar_webcam_colab(conf_threshold=0.7)
processar_imagem("/content/drive/MyDrive/Faculdade_2025/PF/animal/7.jpg")
#processar_imagem("/content/data/arara-azul/5.jpg")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Classe detectada: arara-azul (23.12%)
